In [22]:
model1_weights_dirpath = '/home/maiso/projects/def-emilios/detests_2022/trained_models/BETO20.pt'
model2_weights_dirpath = '/home/maiso/projects/def-emilios/detests_2022/trained_models/roberta15.pt'
model3_weights_dirpath = '/home/maiso/projects/def-emilios/detests_2022/trained_models/roberta17.pt'

#CHANGE
model_no = 3
#CHANGE


model1_name = 'bert-base-spanish-wwm-uncased'
model2_name = 'roberta-base-bne'
model3_name = model2_name
import sys
sys.path.append('/home/maiso/detests_2022/library')



from maiso.task import get_texts, get_y
import numpy as np
from maiso.models import TransformersShell

if model_no==1:
    model = TransformersShell(model_name=model1_name, epochs=1)
    print('model_name='+model1_name)
    model.load_from_pretrained(model1_weights_dirpath)
    print('model_weights='+model1_weights_dirpath)
    name = 'BETO20'
elif model_no==2:
    model = TransformersShell(model_name=model2_name, epochs=1)
    print('model_name='+model2_name)
    model.load_from_pretrained(model2_weights_dirpath)
    print('model_weights='+model2_weights_dirpath)
    name = 'roberta15'
elif model_no==3:
    model = TransformersShell(model_name=model3_name, epochs=1)
    print('model_name='+model3_name)
    model.load_from_pretrained(model3_weights_dirpath)
    print('model_weights='+model3_weights_dirpath)
    name = 'roberta17'

print('name='+name)

Working with model roberta-base-bne
Loading tokenizer...
Loading model...


Some weights of the model checkpoint at /home/maiso/detests_2022/assets/roberta-base-bne/model were not used when initializing RobertaModel: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/maiso/detests_2022/assets/roberta-base-bne/model and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be ab

model_name=roberta-base-bne
model_weights=/home/maiso/projects/def-emilios/detests_2022/trained_models/roberta17.pt
name=roberta17


In [23]:
def make_predictions(model, data):

    # separar cada columna respetando el formato pedido en la competencia
    comment_id_0 = data.comment_id
    sentence_pos_1 = data.sentence_pos
    columns = model.get_columns()
    X_external = data.sentence
    # computar predicciones
    print('Starting predictions')
    preds = model.predecir(X_external)
    preds = preds*1
    print('Finishing predictions')
    
    # computar predicciones para task_1
    # transpongo el arreglo: de (Nx10) pasa a ser de (10xN)
    # luego computo el or, y me queda un arreglo de N predicciones
    task_1 = np.logical_or.reduce(preds.T)*1
    
    # confecciono los dataframes a retornar
    d_1 = {'comment_id': comment_id_0, 'sentence_pos': sentence_pos_1, 'stereotype': task_1 }
    # d_preds = {columns: preds}
    DF_1 = pd.DataFrame(d_1)
    DF_preds = pd.DataFrame(preds, columns=columns)
    DF_2 = pd.concat([DF_1,DF_preds], axis=1)
    return DF_1, DF_2

import pandas as pd
test_data = pd.read_csv('/home/maiso/projects/def-emilios/detests_2022/data/test.csv')

task_1, task_2 = make_predictions(model, test_data)



Starting predictions
Finishing predictions


In [26]:
name = model3_weights_dirpath.split('/')[-1][:-3]
task1_file = f'/home/maiso/projects/def-emilios/detests_2022/results/final_results/best_models/task_1/task1_{name}.csv'
task2_file = f'/home/maiso/projects/def-emilios/detests_2022/results/final_results/best_models/task_2/task2_{name}.csv'

print(task1_file)
print(task2_file)

task_1.to_csv(task1_file, index=False)
task_2.to_csv(task2_file, index=False)

/home/maiso/projects/def-emilios/detests_2022/results/final_results/best_models/task_1/task1_roberta17.csv
/home/maiso/projects/def-emilios/detests_2022/results/final_results/best_models/task_2/task2_roberta17.csv


In [19]:
task_2

,comment_id,sentence_pos,stereotype,xenophobia,suffering,economic,migration,culture,benefits,health,security,dehumanisation,others
0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,0,0,0,0,0,0,0,0,0
2,0,3,0,0,0,0,0,0,0,0,0,0,0
3,0,4,0,0,0,0,0,0,0,0,0,0,0
4,0,5,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,824,4,0,0,0,0,0,0,0,0,0,0,0
1808,824,5,1,0,0,0,1,0,0,0,0,0,0
1809,825,1,0,0,0,0,0,0,0,0,0,0,0
1810,826,1,0,0,0,0,0,0,0,0,0,0,0
